# **M1B1T1_AI3. PROCESAMIENTO DE DATOS CON SPARK 2.X. NATALIDAD EE.UU.**

NOMBRE: ELIZABETH PEREDO MESCCO

In [ ]:
pip install pyspark

In [ ]:
pip install numpy

In [ ]:
pip install panda

In [ ]:
!pip install -U -q PyDrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
from google.colab import files
uploaded = files.upload()

Saving natality.csv to natality.csv


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, desc, avg, mean, column

spark=SparkSession\
    .builder\
    .appName("Test")\
    .getOrCreate()

df=spark.read\
    .option("header","true")\
    .option("inferSchema","true")\
    .csv("natality.csv")
df.printSchema()

df.createOrReplaceTempView("natality")


root
 |-- source_year: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: string (nullable = true)
 |-- wday: integer (nullable = true)
 |-- state: string (nullable = true)
 |-- is_male: boolean (nullable = true)
 |-- child_race: string (nullable = true)
 |-- weight_pounds: double (nullable = true)
 |-- plurality: integer (nullable = true)
 |-- apgar_1min: string (nullable = true)
 |-- apgar_5min: integer (nullable = true)
 |-- mother_residence_state: string (nullable = true)
 |-- mother_race: string (nullable = true)
 |-- mother_age: integer (nullable = true)
 |-- gestation_weeks: integer (nullable = true)
 |-- lmp: integer (nullable = true)
 |-- mother_married: boolean (nullable = true)
 |-- mother_birth_state: string (nullable = true)
 |-- cigarette_use: string (nullable = true)
 |-- cigarettes_per_day: string (nullable = true)
 |-- alcohol_use: string (nullable = true)
 |-- drinks_per_week: string (nullable = true)
 |-- we

# **1. Obtén en qué 10 estados nacieron más bebés en 2003.**

In [ ]:
df.where(df.year==2003)\
    .groupBy("state").count()\
    .sort(desc("count"))\
    .show(10)


+-----+-----+
|state|count|
+-----+-----+
|   PA|   39|
|   WA|   20|
+-----+-----+



In [ ]:
sqlDF = spark.sql(
"""
SELECT state, COUNT(state) 
FROM natality 
WHERE YEAR=2003 
GROUP BY state 
ORDER BY COUNT(state) 
DESC
""")
sqlDF.show()

+-----+------------+
|state|count(state)|
+-----+------------+
|   PA|          39|
|   WA|          20|
+-----+------------+



# **2. Obtén la media de peso de los bebés por año y estado.**

In [ ]:
df.groupBy("year","state").avg("weight_pounds")\
.na.drop(subset=["state"])\
.orderBy("year").show()


+----+-----+------------------+
|year|state|avg(weight_pounds)|
+----+-----+------------------+
|2003|   WA| 7.220139080499999|
|2003|   PA| 7.205867049854738|
|2004|   WA| 7.211556799586427|
|2004|   TN|    7.506188865445|
|2004|   KY| 7.708830427933333|
|2004|   SC|      6.4705673897|
|2004|   PA| 7.104341277384499|
|2004|   NH|     8.24969784404|
|2004|   ID|    6.340053730596|
|2004|   NY|  7.21329067831872|
|2004|   FL| 7.099987147710001|
+----+-----+------------------+



In [ ]:
sqlDF = spark.sql(
"""
SELECT year, state, AVG(weight_pounds) 
FROM natality
WHERE state IS NOT NULL 
GROUP BY year, state 
ORDER BY year ASC
""")
sqlDF.show()

+----+-----+------------------+
|year|state|avg(weight_pounds)|
+----+-----+------------------+
|2003|   WA| 7.220139080499999|
|2003|   PA| 7.205867049854738|
|2004|   WA| 7.211556799586427|
|2004|   TN|    7.506188865445|
|2004|   KY| 7.708830427933333|
|2004|   SC|      6.4705673897|
|2004|   PA| 7.104341277384499|
|2004|   NH|     8.24969784404|
|2004|   ID|    6.340053730596|
|2004|   NY|  7.21329067831872|
|2004|   FL| 7.099987147710001|
+----+-----+------------------+



# **3. Evolución por año y por mes del número de niños y niñas nacidas (Resultado por separado con una sola consulta, cada registro debe tener 4 columnas: año, mes, numero de niños nacidos, numero de niñas nacidas).**


In [ ]:
df.withColumn("Boys",when(df.is_male==True,1).otherwise(0))\
    .withColumn("Girls",when(df.is_male==False,1).otherwise(0))\
    .groupBy("year","month").sum("Boys","Girls")\
    .orderBy("year","month").show()


+----+-----+---------+----------+
|year|month|sum(Boys)|sum(Girls)|
+----+-----+---------+----------+
|2003|    1|        3|         1|
|2003|    3|        0|         4|
|2003|    5|        2|         6|
|2003|    6|        4|         5|
|2003|    7|        2|         4|
|2003|    8|        2|         3|
|2003|    9|        4|         3|
|2003|   10|        5|         2|
|2003|   11|        2|         3|
|2003|   12|        3|         1|
|2004|    1|        4|        10|
|2004|    2|        6|         5|
|2004|    3|        9|         6|
|2004|    4|       10|         6|
|2004|    5|        8|         5|
|2004|    6|       10|        10|
|2004|    7|        9|         6|
|2004|    8|        4|         9|
|2004|    9|        6|         5|
|2004|   10|        6|        11|
+----+-----+---------+----------+
only showing top 20 rows



In [ ]:
sqlDF = spark.sql(
"""
SELECT year, month,SUM(Boys), SUM(Girls) FROM 
(SELECT year, month, 
CASE WHEN is_male = 'true' THEN 1 ELSE 0 END as Boys,
CASE WHEN is_male = 'false' THEN 1 ELSE 0 END as Girls
FROM natality) 
GROUP BY year, month
ORDER BY year,month ASC
"""
)
sqlDF.show()

+----+-----+---------+----------+
|year|month|sum(Boys)|sum(Girls)|
+----+-----+---------+----------+
|2003|    1|        3|         1|
|2003|    3|        0|         4|
|2003|    5|        2|         6|
|2003|    6|        4|         5|
|2003|    7|        2|         4|
|2003|    8|        2|         3|
|2003|    9|        4|         3|
|2003|   10|        5|         2|
|2003|   11|        2|         3|
|2003|   12|        3|         1|
|2004|    1|        4|        10|
|2004|    2|        6|         5|
|2004|    3|        9|         6|
|2004|    4|       10|         6|
|2004|    5|        8|         5|
|2004|    6|       10|        10|
|2004|    7|        9|         6|
|2004|    8|        4|         9|
|2004|    9|        6|         5|
|2004|   10|        6|        11|
+----+-----+---------+----------+
only showing top 20 rows



# **4. Obtén los tres meses de 2005 en que nacieron más bebés.**

In [ ]:
df.where(df.year==2005).groupBy("month").count().sort(desc("count")).show(3)


+-----+-----+
|month|count|
+-----+-----+
|    3|  360|
|   12|  352|
|    8|  350|
+-----+-----+
only showing top 3 rows



In [ ]:
sqlDF = spark.sql(
"""
SELECT month, count(month)
FROM natality 
WHERE year=2005
GROUP BY month
ORDER BY count(month) DESC
LIMIT 3
"""
)
sqlDF.show()

+-----+------------+
|month|count(month)|
+-----+------------+
|    3|         360|
|   12|         352|
|    8|         350|
+-----+------------+



Los meses de marzo, agosto y diciembre del 2005 nacieron más bebés.

# **5. Obtén los estados donde las semanas de gestación son superiores a la media de EE. UU.**

In [ ]:
avgGestation=df.select(avg("gestation_weeks").cast("double")).collect()[0][0]
print("La media de las las semanas de gestación es "+str(avgGestation))

print("""
Cantidad de madres con semanas de 
gestación superiores a la media de EEUU por estado""")
df.where(df.gestation_weeks>avgGestation)\
.groupBy("state").count().na.drop(subset=["state"])\
.sort(desc("count")).show()

print("""
Media de madres con semanas de gestación 
superiores a la media de EEUU por estado""")
df.groupBy("state")\
.agg(avg("gestation_weeks").alias("avg_weeks"))\
.na.drop(subset=["state"])\
.where(col("avg_weeks")>avgGestation)\
.sort(desc("avg_weeks")).show()


La media de las las semanas de gestación es 38.656659939455096

Cantidad de madres con semanas de 
gestación superiores a la media de EEUU por estado
+-----+-----+
|state|count|
+-----+-----+
|   PA|   40|
|   WA|   30|
|   NY|   30|
|   FL|   20|
|   TN|    9|
|   KY|    3|
|   ID|    2|
|   SC|    1|
+-----+-----+


Media de madres con semanas de gestación 
superiores a la media de EEUU por estado
+-----+---------+
|state|avg_weeks|
+-----+---------+
|   KY|     39.0|
+-----+---------+



In [ ]:
print("""
Cantidad de madres con semanas de 
gestación superiores a la media de EEUU por estado""")
sqlDF = spark.sql(
"""
SELECT state, COUNT(state)
FROM natality 
WHERE gestation_weeks>(SELECT AVG(gestation_weeks) FROM natality)
AND state IS NOT NULL
GROUP BY state
ORDER BY count(state) DESC
"""
)
sqlDF.show()


print("""
Media de madres con semanas de gestación 
superiores a la media de EEUU por estado""")
sqlDF = spark.sql(
"""
SELECT * FROM 
(SELECT state, 
AVG(gestation_weeks) AS AVG_WEEKS
FROM natality 
GROUP BY state)
WHERE AVG_WEEKS>(SELECT AVG(gestation_weeks) FROM natality)
AND state IS NOT NULL
ORDER BY AVG_WEEKS DESC
"""
)
sqlDF.show()


Cantidad de madres con semanas de 
gestación superiores a la media de EEUU por estado
+-----+------------+
|state|count(state)|
+-----+------------+
|   PA|          40|
|   WA|          30|
|   NY|          30|
|   FL|          20|
|   TN|           9|
|   KY|           3|
|   ID|           2|
|   SC|           1|
+-----+------------+


Media de madres con semanas de gestación 
superiores a la media de EEUU por estado
+-----+---------+
|state|AVG_WEEKS|
+-----+---------+
|   KY|     39.0|
+-----+---------+



# **6. Obtén los cinco estados donde la media de edad de las madres ha sido mayor.**


In [ ]:
avgAgeMother=df.select(avg("mother_age").cast("int")).collect()[0][0]
print("La edad media de las madres es "+str(avgAgeMother))

df.groupBy("state")\
.agg(avg("mother_age").alias("avg_age"))\
.na.drop(subset=["state"])\
.where(col("avg_age")> avgAgeMother)\
.sort(desc("avg_age")).show(5)


La edad media de las madres es 25
+-----+------------------+
|state|           avg_age|
+-----+------------------+
|   ID|              34.8|
|   KY|33.333333333333336|
|   SC|31.666666666666668|
|   WA|31.346938775510203|
|   PA|31.024390243902438|
+-----+------------------+
only showing top 5 rows



In [ ]:
sqlDF = spark.sql(
"""
SELECT * FROM 
(SELECT state, 
AVG(mother_age) AS AVG_MOTHER_AGE
FROM natality 
GROUP BY state)
WHERE AVG_MOTHER_AGE>(SELECT AVG(mother_age) 
FROM natality)
AND state IS NOT NULL
ORDER BY AVG_MOTHER_AGE DESC
LIMIT 5
"""
)
sqlDF.show()

+-----+------------------+
|state|    AVG_MOTHER_AGE|
+-----+------------------+
|   ID|              34.8|
|   KY|33.333333333333336|
|   SC|31.666666666666668|
|   WA|31.346938775510203|
|   PA|31.024390243902438|
+-----+------------------+



# **7. Indica cómo influye en el peso del bebé y las semanas de gestación que la madre haya tenido un parto múltiple (campo plurality) a las que no lo han tenido.**

In [ ]:
df_plurality=df.groupBy("plurality")\
    .agg(avg("weight_pounds"),avg("gestation_weeks"))\
    .orderBy(column("plurality").desc())\
    .show(truncate=False)

+---------+------------------+--------------------+
|plurality|avg(weight_pounds)|avg(gestation_weeks)|
+---------+------------------+--------------------+
|4        |4.06311948866     |32.5                |
|3        |3.814878981648    |31.733333333333334  |
|2        |4.842073024972972 |34.708708708708706  |
|1        |7.152029712115611 |38.748935895782274  |
+---------+------------------+--------------------+



En la tabla precedente se evidencia que el peso del bebé y las semanas de gestación de la madre tiene una relación inversa con el parto múltiple de la madre. Esto indica que mientras mayor sea el número de hijos, el peso y tiempo de gestación es menor.

In [ ]:
sqlDF = spark.sql(
"""
SELECT plurality, 
AVG(weight_pounds),
AVG(gestation_weeks)
FROM natality 
GROUP BY plurality
ORDER BY plurality DESC
"""
)
sqlDF.show()

+---------+------------------+--------------------+
|plurality|avg(weight_pounds)|avg(gestation_weeks)|
+---------+------------------+--------------------+
|        4|     4.06311948866|                32.5|
|        3|    3.814878981648|  31.733333333333334|
|        2| 4.842073024972972|  34.708708708708706|
|        1| 7.152029712115611|  38.748935895782274|
+---------+------------------+--------------------+

